# "Torsion angle scan with rdkit & xtb"
> "Log of the huzzle doing torsion angle scans with rdkit & xtb - ultimately comparing results to the crystallography open database"

- toc: true
- branch: master
- badges: true
- comments: true
- author: Peter Schmidtke
- categories: [torsion, dihedral, oss, opensource, rdkit, xtb, energy]
- image: images/xtb_torsion_scan.png

Post #4 will be along the lines of dihedral / torsion angle analysis again. The aim is to log some of the hurdles I had to overcome to run a torsion angle analysis with [xtb](https://github.com/grimme-lab/xtb) and / or [rdkit](https://www.rdkit.org/).

What I'm trying to accomplish here is mainly to see how torsion angle scans can be performed - easily and robustly - using open source tools available. This is in preparation of a larger work track on analysing the data from the [COD](http://www.crystallography.net/cod/), where [first steps have already been set here](https://pschmidtke.github.io/blog/rdkit/crystallography/small%20molecule%20xray/xray/database/2021/01/25/cod-and-torsion-angles.html).


## Aim

Given an input molecule and a particular torsion angle I'd like to see what the energy langscape of the molecule looks like when rotating around that torsion angle. I'd like to know how easy/complicated this is using two different tools: 

1. __rdkit__ with the integrated MMFF 
2. __xtb__ from the Grimme lab

The post is also inspired by an older [post done by iwatobipen](https://iwatobipen.wordpress.com/2020/09/06/visualize-the-torsion-drive-with-different-approach-openff-torchani-chemoinformatics-quantum_chemistry/) analyzing openforcefield with Ani2 on some torsion energy predictions using the [torsion drive dataset from the openforcefield initiative](https://qcarchivetutorials.readthedocs.io/en/latest/basic_examples/torsiondrive_datasets.html)

All code used for this post is available on [this separate repo](https://github.com/pschmidtke/dihedral_scans/tree/main/notebooks), as it uses a slightly different environment (you'll see why).

## Dihedral scan with rdkit

I don't really know why, but I started out with this molecule here: 



In [4]:
#collapse-hide

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
mol=Chem.AddHs(Chem.MolFromSmiles('CCCOC1=CC=C(Cl)C(C)=C1'))
for i, a in enumerate(mol.GetAtoms()):
        a.SetAtomMapNum(i)
AllChem.EmbedMolecule(mol,randomSeed=10)    #generate an initial random conformation (randomSeed is fixed to have something reproducible)
#mol
Draw.MolToFile(mol,"../images/mol.png",includeAtomNumbers=True,highlightAtoms=(1,2,3,4))        #sorry - rdkit 2019.03 issues with newer python versions (I guess)

![](../images/mol.png)

The torsion angle I'm interested in is highlighted in red and situated between carbons 1,2,3 & 4. Now let's try to set up a torsion angle scan in rdkit using MMFF (UFF should be similar procedure ... )

In [5]:

from rdkit.Chem import rdMolTransforms
import altair as alt
import copy
import pandas as pd
from rdkit.Chem import rdForceFieldHelpers
from rdkit.Chem import ChemicalForceFields

conformer=mol.GetConformer(0)

m2=copy.deepcopy(mol)
mp2 = AllChem.MMFFGetMoleculeProperties(m2)
energy=[]
confid=0
angles=range(0,370,10)
for angle in angles:
    confid+=1
    ff2 = AllChem.MMFFGetMoleculeForceField(m2, mp2)
    ff2.MMFFAddTorsionConstraint(1,2,3,4, False, angle - .1, angle + .1, 10000.0)
    ff2.Minimize()
    energy.append(ff2.CalcEnergy())

    xyz=ff2.Positions()
    new_conf = Chem.Conformer(mol.GetNumAtoms())
    for i in range(mol.GetNumAtoms()):
        new_conf.SetAtomPosition(i, (m2.GetConformer(-1).GetAtomPosition(i)))
    new_conf.SetId(confid)
    mol.AddConformer(new_conf)


dfrdkit = pd.DataFrame({'angle':angles, 'energy':energy})
alt.Chart(dfrdkit).mark_line(point=True,interpolate="natural").encode(
    alt.X('angle:Q',
        scale=alt.Scale(domain=[0,360,350])
    ),
    alt.Y('energy:Q',
        scale=alt.Scale(zero=False)
    )
).interactive()

alt.Chart(...)

## RDKit issues

Alright, first of all ... to run all of these I have to use rdkit __2019.03.3__ or earlier!! as since then there has been obviously a regression on the MMFF & UFF torsion constrain code (or other). You can [track this issue on github directly](https://github.com/rdkit/rdkit/issues/3781) for newer rdkit versions.
But in the end, this basically means you cannot run any of this with newer rdkit versions, which is suboptimal. 

Next, seeing this plot above, it doesn't look anything like the one I expected from the torsion archive (but a bit better than on the new bugged rdkit, believe me ;)). Here is the one from iwatobipen's post: 

![](https://iwatobipen.files.wordpress.com/2020/09/mol3.png?w=920)




In order to understand what happens here, you really have to look at the minimized conformations. Here's a bit of code to browse through them. I collected all conformations generated with the torsion scan before. 

In [6]:
#collapse-hide

from ipywidgets import interact, interactive, fixed
import py3Dmol
patt = Chem.MolFromSmarts('c1ccccc1');patt
match = mol.GetSubstructMatch(patt)

AllChem.AlignMolConformers(mol,atomIds=match)


def drawit(m,p,confId):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p.removeAllModels()
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    #p.zoomTo()
    return p.show()
viewer = py3Dmol.view(width=500, height=500)
mb = Chem.MolToMolBlock(mol,confId=0)
viewer.addModel(mb,'sdf')
viewer.setStyle({'stick':{}})
viewer.zoomTo()
#viewer.show()
conformerIds=[conf.GetId() for conf in mol.GetConformers()]
interact(drawit, m=fixed(mol),p=fixed(viewer),confId=(0,mol.GetNumConformers()-1))

interactive(children=(IntSlider(value=18, description='confId', max=37), Output()), _dom_classes=('widget-inte…

<function __main__.drawit(m, p, confId)>

You can use the slider above to browse through the first conformers. You'll see that essentially conformers 1 to 5 are wrong. So likely I'm doing something very wrong here (I followed some sample code on that from Jason Biggs posted in the github issue on that) or there's still another bug. Digging a bit deeper into this issue I found that this might come from the out of plane terms of the MMFF in rdkit. To double check that we can actually rerun the whole thing without out of plane energy terms - sorry a bit verbose the code here:  

In [8]:
from rdkit.Chem import rdMolTransforms
import altair as alt
import copy
import pandas as pd
from rdkit.Chem import rdForceFieldHelpers
from rdkit.Chem import ChemicalForceFields

mol=Chem.AddHs(Chem.MolFromSmiles('CCCOC1=CC=C(Cl)C(C)=C1'))
for i, a in enumerate(mol.GetAtoms()):
        a.SetAtomMapNum(i)
AllChem.EmbedMolecule(mol,randomSeed=10) 

conformer=mol.GetConformer(0)
m2=copy.deepcopy(mol)
mp = AllChem.MMFFGetMoleculeProperties(m2)
mp.SetMMFFOopTerm(False)    # That's the critical bit here - switch off out of plane terms for MMFF
ffm = AllChem.MMFFGetMoleculeForceField(m2, mp)
energy=[]
confid=0
angles=range(0,370,10)
for angle in angles:
    confid+=1
    ff2 = AllChem.MMFFGetMoleculeForceField(m2, mp)
    ff2.MMFFAddTorsionConstraint(1,2,3,4, False, angle - .1, angle + .1, 10000.0)
    ff2.Minimize()
    energy.append(ff2.CalcEnergy())
    xyz=ff2.Positions()
    new_conf = Chem.Conformer(mol.GetNumAtoms())
    for i in range(mol.GetNumAtoms()):
        new_conf.SetAtomPosition(i, (m2.GetConformer(-1).GetAtomPosition(i)))
    new_conf.SetId(confid)
    mol.AddConformer(new_conf)

dfrdkit = pd.DataFrame({'angle':angles, 'energy':energy})
alt.Chart(dfrdkit).mark_line(point=True,interpolate="natural").encode(
    alt.X('angle:Q',
        scale=alt.Scale(domain=[0,360,350])
    ),
    alt.Y('energy:Q',
        scale=alt.Scale(zero=False)
    )
).interactive()

alt.Chart(...)

Well, that looks slightly more reasonable in terms of positions of the energy wells. We see a minimum at 180°, other minimas around 75° and 275° & and energy peak around 0°. This is a bit more in line with what we might expect here. You can browse through the conformations below. They look more reasonable as well now: 

In [9]:
#collapse-hide

from ipywidgets import interact, interactive, fixed
import py3Dmol
patt = Chem.MolFromSmarts('c1ccccc1');patt
match = mol.GetSubstructMatch(patt)
AllChem.AlignMolConformers(mol,atomIds=match)
def drawit(m,p,confId):
    mb = Chem.MolToMolBlock(m,confId=confId)
    p.removeAllModels()
    p.addModel(mb,'sdf')
    p.setStyle({'stick':{}})
    return p.show()
viewer = py3Dmol.view(width=500, height=500)
mb = Chem.MolToMolBlock(mol,confId=0)
viewer.addModel(mb,'sdf')
viewer.setStyle({'stick':{}})
viewer.zoomTo()
conformerIds=[conf.GetId() for conf in mol.GetConformers()]
interact(drawit, m=fixed(mol),p=fixed(viewer),confId=(0,mol.GetNumConformers()-1))

interactive(children=(IntSlider(value=18, description='confId', max=37), Output()), _dom_classes=('widget-inte…

<function __main__.drawit(m, p, confId)>

So the bottomline here is at least in this example the out of plane terms are not living well together with my other dihedral constraints. 

## Dihedral scan with XTB

XTB is a toolkit implementing semiempirical quantum mechanics and you can do quite a lot of things. Among these: energy optimization, dihedral scans, constrained optimizations, metadynamics etc...It's conda packaged, so easy to deploy anywhere. 
In contrary to things like Gaussian, Jaguar etc, it's:

1. free
2. opensource
3. actually quite fast

Running an energy optimization with xtb is rather straightforward and would work like this: 

`xtb mol.sdf --opt --charge 0`

This will write the optimized molecule in an SD file, together with the energy (in hartree). 

XTB also supports dihedral scans as described in the documentation and the [examples work well on ethane](https://xtb-docs.readthedocs.io/en/latest/scan.html#ethane). The thing is, as usual ... we are not working with ethane or 1-Bromo-2-chloroethane (the other example). 

Long story short, I tried to integrate a dihedral scan as described in the documentation, but on my molecule above (which still remains rather simple)

This resulted in a ton of segmentation faults (fond memories of Gaussian came back to me) after a few dihedral scan cycles. My suspicion is that the xtb optimizer is not robust enough to allow to resolve really ugly clashes generated during the scan (I'm just guessing here).

So here's the workaround I came up with: preparing "good enough" starting conformations with rdkit and constrained optimizing with xtb and last, gathering the final energy values. Given the issues with the out of plane terms before I would have preferred having a self contained and functioning way to do this with xtb, but well...there are days like these.



In [10]:
import os
angles=range(0,370,10)
xtbenergy=[]

#loop over the previous conformations we obtained with rdkit
for idx,deg in enumerate(angles):
  w = Chem.SDWriter('mol.sdf')
  w.write(mol,confId=idx+1)
  w.close()

  atoms = '2,3,4,5' #set atoms to define the dihedral - NB: xtb indexes start at 1, rdkit at 0
  # Now write the xtb input file:
  fh = open("dihedral_constraint.inp","w")
  fh.write("""$constrain
    force constant=1.00
    dihedral: {},{}
  $end""".format(atoms,float(deg)))
  fh.close()
  # run xtb
  os.system("export OMP_STACKSIZE=48G && export OMP_NUM_THREADS=12,1 && xtb mol.sdf --opt vtight --charge 0 --input dihedral_constraint.inp")

  sdr=Chem.SDMolSupplier("xtbopt.sdf")
  for xtbmol in sdr:
      xtbenergy.append(xtbmol.GetProp("total energy / Eh"))

This runs for a while, but it's still reasonably fast. Also, you might have noticed that I specified an argument to the `--opt` flag. This argument allows you to tweak how loose or precise the optimisation should be. More information on that can be found in the [xtb documentation here](https://xtb-docs.readthedocs.io/en/latest/optimization.html). In this example I specified a rather precise method. Feel free to play around with them and check the outcome (rather interesting as well). 





In [11]:
import numpy as np
angle=np.array(range(0,370,10))

dfxtb = pd.DataFrame({'angle':angle, 'xtb':xtbenergy,'MMFF':dfrdkit["energy"]})

base = alt.Chart(dfxtb).encode(
    alt.X('angle:Q', axis=alt.Axis(title=None),scale=alt.Scale(domain=[0,360,350]))
)
line1 = base.mark_line(stroke='#5276A7', interpolate='natural').encode(
    alt.Y('xtb:Q',
          axis=alt.Axis(title='xtb energy', titleColor='#5276A7'),scale=alt.Scale(zero=False))
)
line2 = base.mark_line(stroke='#57A44C', interpolate='natural').encode(
    alt.Y('MMFF:Q',
          axis=alt.Axis(title='MMFF energy', titleColor='#57A44C'),scale=alt.Scale(zero=False))
)
alt.layer(line1, line2).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)

On this plot we can see both results, from rdkit's MMFF implementation and xtb. Good news...at least they agree on the maximum around 0°. Other than that there are quite important discrepancies on the location of the global minimum and the importance & extent of the energy barriers between them. Interestingly, openFF and ani2 also predict 180° as a global minimum.

## Comparing vs COD

Let's double check these results now [with the initial work done on the COD](https://pschmidtke.github.io/blog/rdkit/crystallography/small%20molecule%20xray/xray/database/2021/01/25/cod-and-torsion-angles.html) (not yet cleaned and curated - so there will be some noise in here still). This is for sure not the only experimental data-source one should use, but I'll define it as my golden source of truth here within the scope of this post. 
First I'll try to identify the smarts patterns from the torsion library that match the dihedral under investigation here: 

In [12]:
patterns=pd.read_table("../data/list_torsion_patterns.txt",header=None,usecols=[1])
selectedPatterns=[]
for torsionSmarts in patterns[1]:
    torsionQuery = Chem.MolFromSmarts(torsionSmarts)
    matches = mol.GetSubstructMatches(torsionQuery)
    if(len(matches)>0):
        if (matches==((1,2,3,4),)):
            selectedPatterns.append(torsionQuery)
            print("selected: ",torsionSmarts)


selected:  [C:1][CX4H2:2]!@;-[OX2:3][c:4]
selected:  [!#1:1][CX4H2:2]!@;-[OX2:3][c:4]
selected:  [C:1][CX4H2:2]!@;-[OX2:3][!#1:4]
selected:  [!#1:1][CX4H2:2]!@;-[OX2:3][!#1:4]
selected:  [!#1:1][CX4:2]!@;-[OX2:3][!#1:4]


Now we have the selected patterns, let's run these through the prepared [COD molecules (a huge local sd file right now)](https://pschmidtke.github.io/blog/rdkit/crystallography/small%20molecule%20xray/xray/database/2021/01/25/cod-and-torsion-angles.html) and gather statistics on angles. NB: the smart patterns used here might be redundant and map the same molecules. So here I'm keeping track of which molecule was previously selected and don't include it in a subsequent calculation anymore:

In [15]:
suppl = Chem.SDMolSupplier('out.sdf',removeHs=False)    #load the COD sd
angles=[]
matchingmols=[]

for pattern in selectedPatterns:
    print(Chem.MolToSmarts(pattern))
    # based on the gist from Geoff Hutchison: https://gist.github.com/ghutchis/b388dd83ddcd7dc0be11f1ed72309da2
    index_map = {}
    for atom in pattern.GetAtoms() :
        map_num = atom.GetAtomMapNum()
        if map_num:
            index_map[map_num-1] = atom.GetIdx()
    map_list = [index_map[x] for x in sorted(index_map)]

    #i=0
    #suppl.reset()
    nMols = len(suppl)
    for i in range(nMols):
        mol=suppl[i]
    #for mol in suppl:
    #    i+=1
        if mol is None: continue
        
        if (i not in matchingmols) :
            conf=mol.GetConformer(0)
            matches = mol.GetSubstructMatches(pattern)
            if(len(matches)>0):
                matchingmols.append(i)
                for match in matches:
                    mapped = [match[x] for x in map_list]
                    angle = rdMolTransforms.GetDihedralDeg(conf, mapped[0],mapped[1],mapped[2],mapped[3])
                    if (angle < 0.0):
                        angle += 360.0
                    angles.append(angle)

[C:1][C&X4&H2:2]!@&-[O&X2:3][c:4]


This runs again for quite some time...so patience is needed. Once this is done, you should see something like that:

In [43]:
#collapse-hide
alt.data_transformers.disable_max_rows()
dfcod = pd.DataFrame({'angles':angles})
alt.Chart(dfcod).mark_bar().encode(
    alt.X("angles:Q", bin=alt.Bin(maxbins=100)),
    y='count()',
)


alt.Chart(...)

In [74]:
#lets convert that to numbers we show in the line-plot together with the previous results

intervals=pd.interval_range(start=0, end=360,periods=37)
codangles=pd.cut(angles, bins=intervals).value_counts()

In [75]:

dfall = pd.DataFrame({'angle':range(0,370,10), 'xtb':xtbenergy,'MMFF':dfrdkit["energy"],'cod':np.array(codangles)})
#print(dfall)
#data = dfxtb.melt('angle')

base = alt.Chart(dfall).encode(
    alt.X('angle:Q', axis=alt.Axis(title=None),scale=alt.Scale(domain=[0,360,350]))
)
line1 = base.mark_line(stroke='#5276A7', interpolate='natural').encode(
    alt.Y('xtb:Q',
          axis=alt.Axis(title='xtb energy', titleColor='#5276A7'),scale=alt.Scale(zero=False))
)
line2 = base.mark_line(stroke='#57A44C', interpolate='natural').encode(
    alt.Y('MMFF:Q',
          axis=alt.Axis(title='MMFF energy', titleColor='#57A44C'),scale=alt.Scale(zero=False))
)
line3 = base.mark_line(stroke='#ff0000', interpolate='natural').encode(
    alt.Y('cod:Q',
          axis=alt.Axis(title='COD angles', titleColor='#ff0000'),scale=alt.Scale(zero=False))
)
alt.layer(line1, line2,line3).resolve_scale(
    y = 'independent'
)

alt.LayerChart(...)

So here we have in red the results from the COD, in green MMFF from rdkit and xtb in blue. Good news is that the positions of the wells seem to be estimated rather well with xtb. MMFF seems slightly off for the preference on 280°. Analysing relative energy differences between local minima, xtb doesn't follow the trend observed using MMFF & results from the COD here. 

In [ ]:
## Conclusion

Initially I wanted to run this on a larger set of molecules, but as usual things turn out to be much less robust than anticipated. So more to come in an upcoming post, on other torsion angles - especially the challenging ones. 

The encouraging aspect here is, that despite all difficulties, one could potentially use this on a larger set of molecules. The instability of xtb on the integrated dihedral scan should be investigated a bit further as well ... essentially to make sure that this is not only due to my totally "noob" use of the toolkit - which is very likely.